In [1]:
#Desafio BB
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import zipfile
!pip install imutils --user
!pip install opencv-python --user
!pip install dlib --user
!pip install sklearn --user

if(not os.path.exists('./set_train') ):
    print('Descompactando dataset')
    with zipfile.ZipFile('./set_train.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
    print('Dataset descompactado!')

caminho_resultado = Path('./resultado')
caminho_resultado.mkdir(exist_ok=True)

Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://atf.intranet.bb.com.br/artifactory/api/pypi/bb-python-local/simple, http://pypi.servicos.bb.com.br/simple
     |████████████████████████████████| 28.7MB 28.6MB/s 
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
caminho_imagens = "./set_train/"

from os import listdir
from os.path import isfile, join
pastas = [f for f in listdir(caminho_imagens) if not isfile(join(caminho_imagens, f))]
#pastas

#
# 0 neutra, 1 feliz, 2 triste, 3 surpreso, 4 bravo
# 
#
# 

In [3]:
#Array que armazena o caminho completo para todos os arquivos
arquivos = []
# sentimentos = ['neutro','feliz','triste','surpreso','bravo']
sentimentos_lista = [0,1,2,3,4]
qnt_amostra = 0
sentimentos=[]

for pasta in pastas:
    caminho_completo = caminho_imagens+pasta+'/bmp/'
    pos_arq_atual = 0
    
    for f in listdir(caminho_completo):
        
        arquivos.append([caminho_completo+f])
        sentimentos.append(sentimentos_lista[pos_arq_atual])
        pos_arq_atual=pos_arq_atual+1
   
    
    pos_arq_atual = 0
    qnt_amostra = qnt_amostra+1
            
if len(arquivos) > 0:
    print('Encontrados: '+str(len(arquivos))+' arquivos')


Encontrados: 130 arquivos


In [4]:
from imutils import face_utils
import dlib
import cv2
 
print('Detectando os pontos da face de todas as imagens')
# p = modelo pré-treinado para reconhecer 68 pontos da face
p = "./lib/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
imagens=[]

for x in range(0,len(arquivos)):
#     print (x)
    image = cv2.imread(arquivos[x][0])
    # image = cv2.imread('time.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)

    # Make the prediction and transfom it to numpy array
    shape = predictor(gray, rects[0])
    shape = face_utils.shape_to_np(shape)
    
    shape_transformado = shape.reshape(-1)
    imagens.append(shape_transformado)
    

print('Detecção concluida')    

Detectando os pontos da face de todas as imagens
Detecção concluida


In [5]:
# Cria conjunto de treino e testes
from sklearn.model_selection import train_test_split


print('Separando amostra de treino e teste')

#Caso queira testar com outras amostras, basta alterar o random state
imagens_treino, imagens_teste, y_treino, y_teste = train_test_split(imagens, sentimentos, test_size=0.25, random_state=100)
print('Separação concluida com '+ str(len(y_treino))+' imagens de treino e '+str(len(y_teste))+' imagens de teste')


y_treino
neutro=0
feliz=0
triste=0
surpreso=0
bravo=0

for item in y_treino:
    if item==0 :
        neutro=neutro+1
    if item==1 :
        feliz=feliz+1
    if item==2 :
        triste=triste+1
    if item==3 :
        surpreso=surpreso+1
    if item==4 :
        bravo=bravo+1

print('Amostra treino:'+str(neutro))
print('Neutro:'+str(neutro))
print('Feliz:'+str(feliz))
print('Triste:'+str(triste))
print('Surpreso:'+str(surpreso))
print('Bravo:'+str(feliz))

y_treino
neutro=0
feliz=0
triste=0
surpreso=0
bravo=0

for item in y_teste:
    if item==0 :
        neutro=neutro+1
    if item==1 :
        feliz=feliz+1
    if item==2 :
        triste=triste+1
    if item==3 :
        surpreso=surpreso+1
    if item==4 :
        bravo=bravo+1
print()
print()
print('Amostra teste:'+str(neutro))
print('Neutro:'+str(neutro))
print('Feliz:'+str(feliz))
print('Triste:'+str(triste))
print('Surpreso:'+str(surpreso))
print('Bravo:'+str(feliz))
    

Separando amostra de treino e teste
Separação concluida com 97 imagens de treino e 33 imagens de teste
Amostra treino:20
Neutro:20
Feliz:16
Triste:19
Surpreso:23
Bravo:16


Amostra teste:6
Neutro:6
Feliz:10
Triste:7
Surpreso:3
Bravo:10


In [6]:
# CNN Grid Search
# O Grid Search ajuda a encontrar hiperparametros de ajuste da CNN, com base
# em um conjunto pré-determinado de variações
# mostrando ao final quais ajustes produziram a melhor acuracidade do modelo
# 
# Obs: também funciona para outros tipos de modelos

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import numpy as np

sc = StandardScaler()
sc.fit(imagens_treino)

imagens_treino_pad = sc.transform(imagens_treino)
imagens_teste_pad = sc.transform(imagens_teste)

clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)


clf.fit(imagens_treino_pad,y_treino)


Iteration 1, loss = 2.19101622
Iteration 2, loss = 2.18151057
Iteration 3, loss = 2.16833375
Iteration 4, loss = 2.15195959
Iteration 5, loss = 2.13293747
Iteration 6, loss = 2.11211009
Iteration 7, loss = 2.08983771
Iteration 8, loss = 2.06664294
Iteration 9, loss = 2.04259204
Iteration 10, loss = 2.01808827
Iteration 11, loss = 1.99360900
Iteration 12, loss = 1.96979420
Iteration 13, loss = 1.94640081
Iteration 14, loss = 1.92388052
Iteration 15, loss = 1.90215504
Iteration 16, loss = 1.88144914
Iteration 17, loss = 1.86113157
Iteration 18, loss = 1.84116261
Iteration 19, loss = 1.82180630
Iteration 20, loss = 1.80311389
Iteration 21, loss = 1.78466742
Iteration 22, loss = 1.76678199
Iteration 23, loss = 1.74984891
Iteration 24, loss = 1.73363468
Iteration 25, loss = 1.71805432
Iteration 26, loss = 1.70312407
Iteration 27, loss = 1.68872601
Iteration 28, loss = 1.67501345
Iteration 29, loss = 1.66212963
Iteration 30, loss = 1.64990059
Iteration 31, loss = 1.63811060
Iteration 32, los

/home/nanny/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=10, learning_rate='constant',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=21, shuffle=True, solver='sgd', tol=1e-09,
              validation_fraction=0.1, verbose=10, warm_start=False)

In [7]:
from joblib import dump, load

#Salva modelo e preparador de dados
dump(clf,'./resultado/MLPGridSearch.joblib')
dump(sc,'./resultado/sc_new.joblib')

y_saida = clf.predict(imagens_teste_pad)
print('Acuracia: %.2f' % accuracy_score(y_teste, y_saida))

Acuracia: 0.39
